In [16]:
import numpy as np
import tensorflow as tf

In [17]:
with open('reviews.txt', 'r') as f:
    reviews = f.read()

In [18]:
reviews[:2000]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   \nstory of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is tu

In [19]:
with open('labels.txt', 'r') as f:
    labels_org = f.read()

In [20]:
from string import punctuation
all_text = ''.join([c for c in reviews if c not in punctuation])
reviews = all_text.split('\n')

all_text = ' '.join(reviews)
words = all_text.split()


In [21]:
len(reviews)

25001

In [22]:
# # Create your dictionary that maps vocab words to integers here
# vocab = set(words)
# vocab_to_int = {w: i for i, w in enumerate(vocab, 1)}
# print(len(vocab_to_int))

# # Convert the reviews to integers, same shape as reviews list, but with integers
# reviews_ints = []
# for r in reviews:
#     ri = [vocab_to_int.get(w) for w in r if vocab_to_int.get(w) is not None]
#     reviews_ints.append(ri)
# reviews_ints[:10]

from collections import Counter
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

reviews_ints = []
for each in reviews:
    reviews_ints.append([vocab_to_int[word] for word in each.split()])

In [23]:
len(reviews_ints)

25001

In [24]:
# Convert labels to 1s and 0s for 'positive' and 'negative'
# print(labels_org)
labels = np.array([1 if l == "positive" else 0 for l in labels_org.split()])
# print(labels)
print(len(labels))

25000


In [25]:
from collections import Counter
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


In [26]:
len(review_lens)

1025

In [27]:
# Filter out that review with 0 length
reviews_ints = [r[0:200] for r in reviews_ints if len(r) > 0]

In [28]:
len(reviews_ints)

25000

In [29]:
from collections import Counter
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 0
Maximum review length: 200


In [30]:
seq_len = 200
features = np.zeros((len(reviews_ints), seq_len), dtype=int)
# print(features[:10,:100])
for i, row in enumerate(reviews_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]
features[:10,:100]
    # features = []
# for r in reviews_ints:
#     features.append(np.pad(r, (0, 200 - len(r)), 'constant').tolist())

# interesting = []
# for i, r in enumerate(reviews_ints):
#     if len(r) < 200:
#         interesting.append(i)

        
# features = np.array([np.pad(r, (len(r), 200-len(r)), 'constant').tolist() for r in reviews_ints])


array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 21025,   308,     6,
            3,  1050,   207,     8,  2138,    32,     1,   171,    57,
           15,    49,    81,  5785,    44,   382,   110,   140,    15,
         5194,    60,   154,     9,     1,  4975,  5852,   475,    71,
            5,   260,    12, 21025,   308,    13,  1978,     6,    74,
         2395],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     

In [31]:
features.shape

(25000, 200)

In [17]:
print(len(reviews_ints[41]))

20


In [32]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16808949925216568313
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 514442338703950407
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 8731839152759907934
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 989200384
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6487222737907903684
physical_device_desc: "device: 0, name: GeForce GTX 750 Ti, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


### train

In [36]:


split_frac = 0.8

split_index = int(split_frac * len(features))

train_x, val_x = features[:split_index], features[split_index:] 
train_y, val_y = labels[:split_index], labels[split_index:]

split_frac = 0.5
split_index = int(split_frac * len(val_x))

val_x, test_x = val_x[:split_index], val_x[split_index:]
val_y, test_y = val_y[:split_index], val_y[split_index:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))
print("label set: \t\t{}".format(train_y.shape), 
      "\nValidation label set: \t{}".format(val_y.shape),
      "\nTest label set: \t\t{}".format(test_y.shape))



			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)
label set: 		(20000,) 
Validation label set: 	(2500,) 
Test label set: 		(2500,)


In [34]:
split_index 

2500

In [38]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0', '/device:XLA_CPU:0', '/device:XLA_GPU:0', '/device:GPU:0']


In [35]:
#------features--------
train_x=train_x[:2000]
train_x.shape
val_x=val_x[:250]
val_x.shape
test_x=test_x[:250]
test_x.shape
#-------labels---------
train_y=train_x[:2000]
train_y.shape
val_y=val_x[:250]
val_y.shape
test_y=test_x[:250]
test_y.shape

(250, 200)

In [40]:
lstm_size = 256
lstm_layers = 2
batch_size = 100
learning_rate = 0.01

In [41]:
n_words = len(vocab_to_int) + 1 # Add 1 for 0 added to vocab

# Create the graph object
tf.reset_default_graph()
with tf.name_scope('inputs'):
    inputs_ = tf.placeholder(tf.int32, [None, None], name="inputs")
    labels_ = tf.placeholder(tf.int32, [None, None], name="labels")
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")

In [42]:

# Size of the embedding vectors (number of units in the embedding layer)
embed_size = 300 

with tf.name_scope("Embeddings"):
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)



In [43]:
embedding[0]

<tf.Tensor 'strided_slice:0' shape=(300,) dtype=float32>

In [44]:
def lstm_cell():
    lstm =tf.contrib.rnn.BasicLSTMCell(lstm_size, reuse=tf.get_variable_scope().reuse)
    return tf.contrib.rnn.DropoutWrapper(lstm,output_keep_prob=keep_prob)



In [45]:
with tf.name_scope("RNN_layers"):
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(lstm_layers)])
    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [46]:
with tf.name_scope("RNN_forward"):
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed, initial_state=initial_state)

In [47]:
%%time
with tf.name_scope('predictions'):
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
    tf.summary.histogram('predictions', predictions)
with tf.name_scope('cost'):
    cost = tf.losses.mean_squared_error(labels_, predictions)
    tf.summary.scalar('cost', cost)

with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

merged = tf.summary.merge_all()

CPU times: user 396 ms, sys: 7.24 ms, total: 404 ms
Wall time: 515 ms


In [48]:
merged

<tf.Tensor 'Merge/MergeSummary:0' shape=() dtype=string>

validation accuracy

In [49]:
with tf.name_scope('validation'):
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

batching

In [50]:
def get_batches(x, y, batch_size=100):
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

training

In [51]:
%%time
epochs = 10

# with graph.as_default():
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('./logs/tb/train', sess.graph)
    test_writer = tf.summary.FileWriter('./logs/tb/test', sess.graph)
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y[:, None],
                    keep_prob: 0.5,
                    initial_state: state}
            summary, loss, state, _ = sess.run([merged, cost, final_state, optimizer], feed_dict=feed)
            #loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            train_writer.add_summary(summary, iteration)
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))
    
            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: y[:, None],
                            keep_prob: 1,
                            initial_state: val_state}
                    #batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    summary, batch_acc, val_state = sess.run([merged, accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
            test_writer.add_summary(summary, iteration)
            saver.save(sess, "checkpoints/sentiment_manish.ckpt")
    saver.save(sess, "checkpoints/sentiment_manish.ckpt")

Epoch: 0/10 Iteration: 5 Train loss: 0.346
Epoch: 0/10 Iteration: 10 Train loss: 0.248
Epoch: 0/10 Iteration: 15 Train loss: 0.247
Epoch: 0/10 Iteration: 20 Train loss: 0.257
Epoch: 0/10 Iteration: 25 Train loss: 0.236
Val acc: 0.587
Epoch: 0/10 Iteration: 30 Train loss: 0.281
Epoch: 0/10 Iteration: 35 Train loss: 0.271
Epoch: 0/10 Iteration: 40 Train loss: 0.255
Epoch: 0/10 Iteration: 45 Train loss: 0.255
Epoch: 0/10 Iteration: 50 Train loss: 0.262
Val acc: 0.527
Epoch: 0/10 Iteration: 55 Train loss: 0.247
Epoch: 0/10 Iteration: 60 Train loss: 0.243
Epoch: 0/10 Iteration: 65 Train loss: 0.241
Epoch: 0/10 Iteration: 70 Train loss: 0.318
Epoch: 0/10 Iteration: 75 Train loss: 0.276
Val acc: 0.496
Epoch: 0/10 Iteration: 80 Train loss: 0.264
Epoch: 0/10 Iteration: 85 Train loss: 0.258
Epoch: 0/10 Iteration: 90 Train loss: 0.274
Epoch: 0/10 Iteration: 95 Train loss: 0.267
Epoch: 0/10 Iteration: 100 Train loss: 0.272
Val acc: 0.496
Epoch: 0/10 Iteration: 105 Train loss: 0.251
Epoch: 0/10 Ite

Epoch: 4/10 Iteration: 865 Train loss: 0.019
Epoch: 4/10 Iteration: 870 Train loss: 0.030
Epoch: 4/10 Iteration: 875 Train loss: 0.005
Val acc: 0.588
Epoch: 4/10 Iteration: 880 Train loss: 0.034
Epoch: 4/10 Iteration: 885 Train loss: 0.016
Epoch: 4/10 Iteration: 890 Train loss: 0.018
Epoch: 4/10 Iteration: 895 Train loss: 0.029
Epoch: 4/10 Iteration: 900 Train loss: 0.028
Val acc: 0.734
Epoch: 4/10 Iteration: 905 Train loss: 0.012
Epoch: 4/10 Iteration: 910 Train loss: 0.023
Epoch: 4/10 Iteration: 915 Train loss: 0.023
Epoch: 4/10 Iteration: 920 Train loss: 0.025
Epoch: 4/10 Iteration: 925 Train loss: 0.009
Val acc: 0.592
Epoch: 4/10 Iteration: 930 Train loss: 0.012
Epoch: 4/10 Iteration: 935 Train loss: 0.002
Epoch: 4/10 Iteration: 940 Train loss: 0.002
Epoch: 4/10 Iteration: 945 Train loss: 0.002
Epoch: 4/10 Iteration: 950 Train loss: 0.001
Val acc: 0.618
Epoch: 4/10 Iteration: 955 Train loss: 0.001
Epoch: 4/10 Iteration: 960 Train loss: 0.000
Epoch: 4/10 Iteration: 965 Train loss: 0

Val acc: 0.742
Epoch: 8/10 Iteration: 1705 Train loss: 0.035
Epoch: 8/10 Iteration: 1710 Train loss: 0.029
Epoch: 8/10 Iteration: 1715 Train loss: 0.042
Epoch: 8/10 Iteration: 1720 Train loss: 0.019
Epoch: 8/10 Iteration: 1725 Train loss: 0.021
Val acc: 0.512
Epoch: 8/10 Iteration: 1730 Train loss: 0.019
Epoch: 8/10 Iteration: 1735 Train loss: 0.074
Epoch: 8/10 Iteration: 1740 Train loss: 0.117
Epoch: 8/10 Iteration: 1745 Train loss: 0.150
Epoch: 8/10 Iteration: 1750 Train loss: 0.105
Val acc: 0.696
Epoch: 8/10 Iteration: 1755 Train loss: 0.043
Epoch: 8/10 Iteration: 1760 Train loss: 0.213
Epoch: 8/10 Iteration: 1765 Train loss: 0.147
Epoch: 8/10 Iteration: 1770 Train loss: 0.172
Epoch: 8/10 Iteration: 1775 Train loss: 0.100
Val acc: 0.636
Epoch: 8/10 Iteration: 1780 Train loss: 0.100
Epoch: 8/10 Iteration: 1785 Train loss: 0.178
Epoch: 8/10 Iteration: 1790 Train loss: 0.105
Epoch: 8/10 Iteration: 1795 Train loss: 0.093
Epoch: 8/10 Iteration: 1800 Train loss: 0.097
Val acc: 0.608
Epoch

test

In [52]:


test_acc = []
with tf.Session() as sess:
    saver.restore(sess, "checkpoints/sentiment_manish.ckpt")
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
                labels_: y[:, None],
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))



INFO:tensorflow:Restoring parameters from checkpoints/sentiment_manish.ckpt
Test accuracy: 0.558
